In [3]:
## Install all dependencies
# !pip install gensim
# !pip install nltk
# !pip install pandas
# !pip install numpy
# !pip install requests
#!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.6 MB/s eta 0:00:00a 0:00:01


In [1]:
# Import libraries
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
import pandas as pd
import numpy as np
import PyPDF2
import re
import plotly.graph_objects as go
import nltk
nltk.download('punkt')

ModuleNotFoundError: No module named 'gensim'

In [2]:
# Load data
df = pd.read_csv('./dataset/nyc-jobs-1.csv')
# Check data
df.head()

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,87990,DEPARTMENT OF BUSINESS SERV.,Internal,1,Account Manager,CONTRACT REVIEWER (OFFICE OF L,40563,1,,,...,"Salary range for this position is: $42,405 - $...",,,,NaN,New York City residency is generally required ...,2011-06-24T00:00:00,NaN,2011-06-24T00:00:00,2018-07-17T00:00:00
1,97899,DEPARTMENT OF BUSINESS SERV.,Internal,1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,10009,M3,,F,...,,"In addition to applying through this website, ...",,,NaN,New York City residency is generally required ...,2012-01-26T00:00:00,NaN,2012-01-26T00:00:00,2018-07-17T00:00:00
2,102221,DEPT OF ENVIRONMENT PROTECTION,External,1,Project Specialist,ENVIRONMENTAL ENGINEERING INTE,20616,0,,F,...,Appointments are subject to OMB approval,click the apply now button,35 hours per week/day,,NaN,New York City Residency is not required for th...,2012-06-21T00:00:00,NaN,2012-09-07T00:00:00,2018-07-17T00:00:00
3,102221,DEPT OF ENVIRONMENT PROTECTION,Internal,1,Project Specialist,ENVIRONMENTAL ENGINEERING INTE,20616,0,,F,...,Appointments are subject to OMB approval,click the apply now button,35 hours per week/day,,NaN,New York City Residency is not required for th...,2012-06-21T00:00:00,NaN,2012-09-07T00:00:00,2018-07-17T00:00:00
4,114352,DEPT OF ENVIRONMENT PROTECTION,Internal,5,Deputy Plant Chief,SENIOR STATIONARY ENGINEER (EL,91639,0,,F,...,Appointments are subject to OMB approval Fo...,"Click ""Apply Now"" button",40 per week / day,Various,NaN,New York City residency is generally required ...,2012-12-12T00:00:00,NaN,2012-12-13T00:00:00,2018-07-17T00:00:00


In [3]:
# Show column name
df.columns

Index(['Job ID', 'Agency', 'Posting Type', '# Of Positions', 'Business Title',
       'Civil Service Title', 'Title Code No', 'Level', 'Job Category',
       'Full-Time/Part-Time indicator', 'Salary Range From', 'Salary Range To',
       'Salary Frequency', 'Work Location', 'Division/Work Unit',
       'Job Description', 'Minimum Qual Requirements', 'Preferred Skills',
       'Additional Information', 'To Apply', 'Hours/Shift', 'Work Location 1',
       'Recruitment Contact', 'Residency Requirement', 'Posting Date',
       'Post Until', 'Posting Updated', 'Process Date'],
      dtype='object')

In [4]:
df =df[['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills']]
df.head()

,Business Title,Job Description,Minimum Qual Requirements,Preferred Skills
0,Account Manager,Division of Economic & Financial Opportunity (...,1.\tA baccalaureate degree from an accredited ...,â€¢\tExcellent interpersonal and organizationa...
1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",The New York City Department of Small Business...,1. A baccalaureate degree from an accredited c...,
2,Project Specialist,"Under direct supervision, perform elementary e...",A Baccalaureate degree from an accredited coll...,
3,Project Specialist,"Under direct supervision, perform elementary e...",A Baccalaureate degree from an accredited coll...,
4,Deputy Plant Chief,"Under general direction, is in responsible cha...",1. Six years of full-time satisfactory experie...,


In [5]:
df['data'] = df[['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

df.drop(['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills'], axis=1, inplace=True)

print(df.head())

                                                data
0  Account Manager Division of Economic & Financi...
1  EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT The N...
2  Project Specialist Under direct supervision, p...
3  Project Specialist Under direct supervision, p...
4  Deputy Plant Chief Under general direction, is...


In [6]:
# Tag data
data = list(df['data'])
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]

#### Model initialization and vocabulary buiding


**Parameters** of Doc2Vec are as follows: 

- `vector_size`: Dimensionality of the feature vectors. Default: 100.
- `window`: The window refers to the maximum distance between the target word and its context words within a sentence. Default: 5.
- `min_count`: Ignores all words with a total frequency lower than this. Default: 5.
- `epochs`: Number of iterations (epochs) over the corpus. Defaults to 5 for PV-DBOW and 10 for PV-DM.
- `dm`: Defines the training algorithm. If `dm = 1`, the Distributed Memory (PV-DM) model is used. If `dm = 0`, the Distributed Bag of Words (PV-DBOW) model is used. Default: 1 (PV-DM).
- `dbow_words`: If set to 1, trains word vectors (in addition to document vectors) using the PV-DBOW algorithm. Default: 0 (False).
- `dm_mean`: If set to 1, uses the mean of the context word vectors instead of concatenation when inferring vectors in the PV-DM model. Default: 0 (False).
- `dm_concat`: If set to 1, concatenates the document and context word vectors when inferring vectors in the PV-DM model. Default: 0 (False).
- `dm_tag_count`: Expected number of document tags per document, when using the PV-DM algorithm. Default: 1.
- `dbow_tag_count`: Expected number of document tags per document, when using the PV-DBOW algorithm. Default: 1.
- `alpha`: The initial learning rate. Default: 0.025.
- `min_alpha`: The learning rate will linearly drop to `min_alpha` as training progresses. Default: 0.0001.
- `hs`: If set to 1, hierarchical softmax activation function will be used. Default: 0 (Negative Sampling).
- `negative`: If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drawn. Default: 5.
- `ns_exponent`: The exponent used to shape the negative sampling distribution. Default: 0.75.


In [7]:
# Model initialization
model = Doc2Vec(vector_size = 50,
min_count = 5,
epochs = 100,
alpha = 0.001
)

In [8]:
# Vocabulary building
model.build_vocab(tagged_data)
# Get the vocabulary keys
keys = model.wv.key_to_index.keys()
# Print the length of the vocabulary keys
print(len(keys))

8599


In [53]:
# Train the model
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
    model.train(tagged_data, 
                total_examples=model.corpus_count, 
                epochs=model.epochs)

model.save('cv_job_maching.model')
print("Model saved")

Training epoch 1/100
Training epoch 2/100
Training epoch 3/100
Training epoch 4/100
Training epoch 5/100
Training epoch 6/100
Training epoch 7/100
Training epoch 8/100
Training epoch 9/100
Training epoch 10/100
Training epoch 11/100
Training epoch 12/100
Training epoch 13/100
Training epoch 14/100
Training epoch 15/100
Training epoch 16/100
Training epoch 17/100
Training epoch 18/100
Training epoch 19/100
Training epoch 20/100
Training epoch 21/100
Training epoch 22/100
Training epoch 23/100
Training epoch 24/100
Training epoch 25/100
Training epoch 26/100
Training epoch 27/100
Training epoch 28/100
Training epoch 29/100
Training epoch 30/100
Training epoch 31/100
Training epoch 32/100
Training epoch 33/100
Training epoch 34/100
Training epoch 35/100
Training epoch 36/100
Training epoch 37/100
Training epoch 38/100
Training epoch 39/100
Training epoch 40/100
Training epoch 41/100
Training epoch 42/100
Training epoch 43/100
Training epoch 44/100
Training epoch 45/100
Training epoch 46/1

In [2]:
pdf = PyPDF2.PdfReader('CV/Shubendu_october.pdf')
resume = ""
for i in range(len(pdf.pages)):
    pageObj = pdf.pages[i]
    resume += pageObj.extract_text()

In [29]:
jd = """ArithmeticErrorHelp architect and build out our web app backend
Work with the front end team to create API interfaces
Lead the way in developing scalable microservices
Work on advanced financial forecasting algorithms and ML
Upgrade our infrastructure to enterprise grade security
Write reusable, testable, and efficient code
Design and implement of low-latency, high-availability, and performant applications
Integration of user-facing elements developed by front-end developers with server side logic
Implementation of security and data protection
Integration of data storage solutions
Performance tuning, improvement, balancing, usability, automation"""

In [8]:
jd = """
Job Description - Senior Data Scientist
About Us
We turn customer challenges into growth opportunities.
Material is a global strategy partner to the world’s most recognizable brands and innovative
companies. Our people around the globe thrive by helping organizations design and deliver
rewarding customer experiences.
We use deep human insights, design innovation and data to create experiences powered by modern
technology. Our approaches speed engagement and growth for the companies we work with and
transform relationships between businesses and the people they serve.
Srijan, a Material company, is a renowned global digital engineering firm with a reputation for solving
complex technology problems using their deep technology expertise and leveraging strategic
partnerships with top-tier technology partners.
What you will get:
Competitive Salaries with flexi benefits
Group Mediclaim Insurance and Personal Accidental Policy
30+ Paid Leaves in a year
Learning and Development of quarterly budgets for certification
Job Summary:
We are seeking a dynamic Senior Data Scientist for our data science and engineering initiatives
within the media domain. This pivotal role requires an experienced professional capable of
navigating the intricacies of the end-to-end data science and modeling lifecycle, having a strong
problem-solving ability and a knack for statistical analysis along with a strong emphasis on Machine
Learning Operations (MLOps) practices. If you’re also able to align our data products with our
business goals, we’d like to meet you.
Position Overview:
We are actively looking for a seasoned Senior Data Scientist to drive our advanced data science and
engineering initiatives. This influential role demands a proficient professional adept at steering the
complexities of the complete data science and modeling lifecycle. The ideal candidate will possess a
robust problem-solving acumen, excel in statistical analysis, ML, Deep Learning, and demonstrate a
keen focus on implementing best-in-class Machine Learning Operations (MLOps) practices. If you
possess the ability to seamlessly align data products with overarching business objectives, we eagerly
await the opportunity to connect with you.
Responsibilities:●
Demonstrated expertise with 3-5 years of hands-on experience in AI/ML, NLP, Python, and
cloud platforms such as AWS/Azure/GCP.
Proven proficiency in advanced technologies including Computer Vision, Machine Vision,
Speech Recognition, and Docker.
Actively drive the ideation, research, and development of sophisticated machine learning
models for media content analysis, employing cutting-edge techniques such as natural
language processing, computer vision, speech recognition, and deep learning.
Follow MLOps principles to ensure scalability, repeatability, and automation in the
end-to-end machine learning lifecycle.
Exhibit agility by rapidly prototyping solutions and building models to assess the feasibility of
various solution approaches.
Develop robust, scalable, and production-grade data pipelines and machine learning models
to address key business problems, regardless of the volume or diversity of data sets.
Construct scalable AI/ML solutions, tailored to specific requirements, spanning NLP,
Computer/Machine Vision, Speech Recognition, Clustering, Segmentation, and Deep
Learning.
Optimize models for performance, taking into consideration real-world constraints and
requirements.
Write reusable code following best software engineering practices, accommodating common
design patterns and Object-Oriented Programming (OOPs) principles.
Follow coding standards, documentation practices, and model version control within the
team.
Work closely with cross-functional teams, including engineering, product management, and
content creation, to gather requirements and deliver impactful, technically sound solutions.
Must Have Skills:
3+ years of relevant data science experience in a delivery focused team
Strong coding ability in Python, familiarity with relevant libraries (e.g., TensorFlow, PyTorch),
and general software development skills (source code management, debugging, testing,
deployment, etc.) is a must
Knowledge of both traditional as well awareness with existing state of the art methods in
Machine Learning, NLP, Computer Vision, Speech Recognition, Transfer Learning, Text
Analysis, Information Extraction and Retrieval with a proven track record of applying them to
real problems.
Experience with end-to-end model deployment and orchestration in production
environments.
Experience in cloud computing like AWS is a plus
Experience working with microservice based architectures and Docker
Strong team player; ability to provide both a strong individual contribution but also work as a
team and contribute to wider goals is a must in this dynamic environment
Strong verbal and written communication skills to establish proper communications within
the team and clients
"""

In [10]:
jd = """
Proactively engage with clients in the branch and deliver an outstanding service experience through completing transactions and by identifying opportunities for advice, solutions, digital enablement and partner introductions
Focus on education and demonstration, leverage technology to deliver a memorable client experience, drive solutions and retain business
Contribute to team results by listening and spotting opportunities to offer additional advice, introduce clients to the capability of RBC partners, or personally fulfil client solutions
Proactively take ownership of resolving and preventing client banking problems
Cultivate and maintain relationships with partners to work as one RBC team
Manage risks by adhering to compliance routines, processes, and controls to protect client and shareholder interests while completing transactions
What do you need to succeed?

Must-have

Goal-oriented individual with a demonstrated passion for putting clients first.
Drive and self-motivation, as well as excellent communication skills and emotional intelligence
Digital literacy across a broad range of devices (i.e., smartphones, tablets, laptops, etc.)
Personal flexibility to work flex hours
Eagerness to learn and determination to succeed
Confidence and ability to learn financial concepts and willingness to obtain the Investment Funds in Canada or the Canadian Securities Course
Nice-to-have

Track record in building rapport and maintaining client relationships within the financial, service or retail industry
Mutual Funds accreditation
Is this job right for you? Check out our video and decide for yourself!

What’s in it for you?

We thrive on the challenge to be our best, progressive thinking to keep growing, and working together to deliver trusted advice to help our clients thrive and communities prosper. We care about each other, reaching our potential, making a difference to our communities, and achieving success that is mutual.

A comprehensive Total Rewards Program including bonuses and flexible benefits, competitive compensation, commissions, and stock where applicable
A world-class training program in financial services
Excellent career development and access to a variety of job opportunities across business and geographies
Leaders who support your development through coaching and managing opportunities
Work in a dynamic, collaborative, progressive, and high-performing team
We also strive to provide an accessible candidate experience for our prospective employees with different abilities. Please let us know if you need any accommodations during the recruitment process.

Join our Talent Community

Stay in-the-know about great career opportunities at RBC. Sign up and get customized info on our latest jobs, career tips and Recruitment events that matter to you.

Expand your limits and create a new future together at RBC. Find out how we use our passion and drive to enhance the well-being of our clients and communities at rbc.com/careers.
"""

In [9]:
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()
    
    # Remove punctuation from the text
    text = re.sub('[^a-z]', ' ', text)
    
    # Remove numerical values from the text
    text = re.sub(r'\d+', '', text)
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return text
# Apply to CV and JD
input_CV = preprocess_text(resume)
input_JD = preprocess_text(jd)

# Model evaluation
model = Doc2Vec.load('cv_job_maching.model')
v1 = model.infer_vector(input_CV.split())
v2 = model.infer_vector(input_JD.split())
similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
print(round(similarity, 2))

58.02


In [10]:
input_CV = """shubendu biswas chandigarh in biswasshubendu gmail com linkedin portfolio github summary with over years of experience i excel as a highly skilled data scientist specializing in ai engineering particularly in generative ai nlp and machine learning i have a demonstrated history of deploying sophisticated ml models in production settings and exhibit adeptness in both django rest framework drf and flask rest apis experience data engineer data scientist april october grazitti interactive quark city mohali in accomplished efficient data retrieval from a community platform by hitting apis daily using cron jobs employed python and pandas for data transformation and successfully stored the processed data in bigquery utilized nlp techniques to analyze user comments and determine sentiment enhancing insights for further decision making successfully deployed fine tuned llm models llama and mistral using hugging face flask api and docker on aws ec achieved a remarkable cost reduction and latency reduction by employing quantized versions of the models advanced utilization of hugging face transformers and open source nlp nlg modules to architect sophisti cated language models and innovative applications leveraging pyspark created data pipeline using aws glue processed semi structured data from marketo apis seamlessly integrating it into postgresql database which reduced manual workload by monthly tech python machine learning gcp aws glue s ec lambda llm huggingface nlp data pipelines pyspark generative ai machine learning engineer january march generativeapis noida in designed and developed restful apis using django rest framework seamlessly integrating and deploying deep learning endpoints to enable advanced computer vision tasks optimized database performance with redis caching reducing query response times by and implemented celery for distributed processing which increased in system scalability implemented nginx as a reverse proxy and load balancer for drf apis significantly boosting the platform s capacity to handle more concurrent requests utilized docker for streamlined application deployment and effectively managed aws ec instances achieving cost savings through efficient resource utilization tech python deep learning nginx docker s ec celery redis mysql orm selenium machine learning engineer intern july december generativeapis noida in designed and developed image scrapper created a flask api utilizing selenium to scrape images from a designated website subsequently storing them in s managed the storage of all associated metadata in an sqlite database skills languages and libraries python c sql tensorflow pytorch pyspark drf flask api technologies and frameworks numpy pandas ocr nlp docker nginx celery redis kubernetes cloud and ai technologies aws ec glue etl s sagemaker gcp huggingface llmprojects amazon fashion recommendation engine created cnn based model for image feature extraction and compared using euclidean distance also used bow and avg w v tfidf on metadata of images built using django deployed on heroku github mahindra first choice services identifying car ownership patterns and spending behavior across the country analyzed car make model service time and type for location specific insights employed arima time series analysis to predict customer lifetime value and revenue forecast github youtube bookmark playlist using django i implemented login authentication for secure user access utilized postgresql with the django orm for efficient database management and performed all crud operations seamlessly integrated ajax for improved user experience and dynamic interactions github sentiment analysis of amazon fine food reviews nlp based text data preprocessing with word vec conversion utilized supervised ml algorithms knn logistic regression svm naive bayes decision tree random forest xgboost with auc roc metric hyperparameter tuning applied unsupervised techniques svd pca t sne k means clustering for data exploration github education master of computer applications jaipur in july july manipal university jaipur full stack data science engineering mumbai in july nov grey atom school of data science bachelor of computer applications hons jalandhar in july june lovely professional university awards and extra curricular activities achieved st place in the chatgpt hackathon at grazitti interactive surpassing participants showcasing top notch innovation and problem solving abilities recognized as the best employee for outstanding contributions and dedication setting a benchmark for excellence and commitment within the organization won first place in machine learning hackathon conducted by grey atom mumbai jan successfully completed coursework on natural language processing with deep learning certificate conducted by grey atom mumbai december actively writing blog posts about machine learning and deep learning sota techniques"""

# Model evaluation
model = Doc2Vec.load('cv_job_maching.model')
v1 = model.infer_vector(input_CV.split())
v2 = model.infer_vector(input_JD.split())
similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
print(round(similarity, 2))

58.02


In [5]:
# Model evaluation
model = Doc2Vec.load('cv_job_maching.model')
v1 = model.infer_vector(input_CV.split())
v2 = model.infer_vector(input_JD.split())
similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
print(round(similarity, 2))

58.02


#### 8. Visualization and Notification

In [6]:
# Visualization
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = similarity,
    mode = "gauge+number",
    title = {'text': "Matching percentage (%)"},
    #delta = {'reference': 100},
    gauge = {
        'axis': {'range': [0, 100]},
        'steps' : [
            {'range': [0, 50], 'color': "#FFB6C1"},
            {'range': [50, 70], 'color': "#FFFFE0"},
            {'range': [70, 100], 'color': "#90EE90"}
        ],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 100}}))

fig.update_layout(width=600, height=400)  # Adjust the width and height as desired
fig.show()

# Print notification
if similarity < 50:
    print(colored("Low chance, need to modify your CV!", "red", attrs=["bold"]))
elif similarity >= 50 and similarity < 70:
    print(colored("Good chance but you can improve further!", "yellow", attrs=["bold"]))
else:
    print(colored("Excellent! You can submit your CV.", "green", attrs=["bold"]))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed